# Algorithmes Évolutionnaires — Travaux dirigés N◦6 : neuroévolution

Dans cette séance, nous allons développer un algorithme évolutionnaire du type NEAT pour optimiser un réseau de neurones. Pour ce faire, nous utiliserons les frameworks DEAP et Keras.

### 1 Introduction
Pour appliquer **les algorithmes évolutionnaires aux réseaux de neurones**, il y a deux possibilités : soit on code soi-même les primitives du réseau de neurones, soit on s’appuye sur un frameworks, comme **Keras**, qui fournit toutes ces primitives et plein de choses en plus. 
Dans les deux cas, il y a des avantages et des inconvénients : 
* si on décide de coder soi-même, il faudra investir du temps dans le développement, mais on aura un contrôle et une compréhension parfaite sur le code produit ; 
* si on opte pour un framework, il faudra investir du temps pour apprendre à l’utiliser, mais on aura accès à tout ce qu’il y a de plus récent et performant en matière d’apprentissage profond

-------------------------------------------------------------- 

class = Variable de classe (1:test positif pour le diabète, 0 : test négatif pour le diabète)


### NeuroEvolution of Augmenting Topologies (NEAT)
- Démarrer avec des topologies aléatoires minimales.
- Augmenter les topologies au fur et à mesure si nécessaire.
- Suivez les gènes correspondants pour atténuer le problème des conventions concurrentes.
- Protéger les innovations par la spéciation.

<font color='green'> **La fonction de fitness** sera basée sur la **fonction de perte** du réseau de neurones.
Une fonction de perte, ou **Loss function**, est une fonction qui **évalue l’écart entre les prédictions réalisées par le réseau de neurones et les valeurs réelles des observations utilisées pendant l’apprentissage**. Plus le résultat de cette fonction **est minimisé**, plus le réseau de neurones **est performant**. Sa minimisation, c’est-à-dire réduire au minimum l’écart entre la valeur prédite et la valeur réelle pour une observation donnée, se fait en ajustant les différents poids du réseau de neurones.</font>

**Compute the Brier score loss**

Plus la perte du score de Brier est faible, mieux c'est, d'où l'appellation de "perte". Le score de Brier mesure la différence moyenne au carré entre la probabilité prédite et le résultat réel (mean squared difference). Le score de Brier prend toujours une valeur comprise entre zéro et un, car il s'agit de la plus grande différence possible entre une probabilité prédite (qui doit être comprise entre zéro et un) et le résultat réel (qui ne peut prendre que des valeurs comprises entre 0 et 1). Il peut être décomposé en la somme de la perte de raffinement et de la perte de calibration.

Le score de Brier est approprié pour les résultats **binaires** et catégoriels qui peuvent être structurés comme **vrai ou faux**, mais il est inapproprié pour les variables ordinales qui peuvent prendre trois valeurs ou plus (ceci parce que le score de Brier suppose que tous les résultats possibles sont équivalemment "distants" les uns des autres). L'étiquette qui est considérée comme l'étiquette positive est contrôlée par le paramètre pos_label, qui prend par défaut l'étiquette la plus grande, sauf si y_true est tout à fait 0 ou tout à fait -1, auquel cas pos_label prend par défaut la valeur 1.

RAPPEL DES CONSTANTES DE DEPART
------------------------------------------------------------------------

* **generation** = Nombre maximum de générations = nombre max de population de réseaux de neurones testées (=> epoch = 10)
* **population** = Taille de la population = Nombre de réseau de neurones dans la population (population = 10)
* **mut** = Probabilité d'une mutation (<= .05)
* **crossover** = Probabilité d'une recombination (Crossover: top 5 randomly select 2 partners)

RAPPEL DES OPERATEURS 
-----------------------------------------------------------
* Mutation : NEAT Structural Mutation 
    - Ajout de neurones
    - Ajout de connexions
* Recombinaison : NEAT Crossover 
    - Création d'un nouveau réseau de neurones à partir de deux réseaux parents
* Fitness : Loss Function 
    - Minimiser la perte 



In [16]:
import random
import logging
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import brier_score_loss

logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s %(message)s',
                    handlers=[logging.FileHandler("ann_test.log"),
                              logging.StreamHandler()])

class ANN(Sequential):
    
    def __init__(self, child_weights=None):
        super().__init__()

        if child_weights is None:
            layer1 = Dense(8, input_shape=(8,), activation='sigmoid')
            layer2 = Dense(1, activation='sigmoid')
            self.add(layer1)
            self.add(layer2)
        else:
            self.add(
                Dense(
                    8,
                    input_shape=(8,),
                    activation='sigmoid',
                    weights=[child_weights[0], np.ones(8)])
                )
            self.add(
                Dense(
                    1,
                    activation='sigmoid',
                    weights=[child_weights[1], np.zeros(1)])
            )

    def forward_propagation(self, train_feature, train_label):
        predict_label = self.predict(train_feature.values)
        # la fitness est 1/(1+loss), ainsi plus la loss est petite, plus la fitness est grande
        self.fitness = 1/(1+ brier_score_loss(train_label, predict_label.round()))
        #self.fitness = 1/(1 + log_loss(train_label, predict_label.round()))
        # self.fitness = accuracy_score(train_label, predict_label.round())


def crossover(nn1, nn2):
    
    nn1_weights = []
    nn2_weights = []
    child_weights = []

    for layer in nn1.layers:
        nn1_weights.append(layer.get_weights()[0])

    for layer in nn2.layers:
        nn2_weights.append(layer.get_weights()[0])

    for i in range(len(nn1_weights)):
        # Get single point to split the matrix in parents based on # of cols
        split = random.randint(0, np.shape(nn1_weights[i])[1]-1)
        # Iterate through after a single point and set the remaing cols to nn_2
        for j in range(split, np.shape(nn1_weights[i])[1]-1):
            nn1_weights[i][:, j] = nn2_weights[i][:, j]

        child_weights.append(nn1_weights[i])

    mutation(child_weights)

    child = ANN(child_weights)
    return child

def mutation(child_weights):
    selection = random.randint(0, len(child_weights)-1)
    mut = random.uniform(0, 1)
    if mut <= .05:
        child_weights[selection] *= random.randint(2, 5)
    else:
        pass

# Preprocess Data
df = pd.read_table('./diabetes.txt',header=None,encoding='gb2312',sep='\t')
df.astype(float)
# remove redundant col which is the opposite value of the 10th col
df.pop(10)
# remove first col of bias = 1
df.pop(0)
# the label column
label = df.pop(9)
# train feature
train_feature = df[:576]
# train label
train_label = label[:576]
# test feature
test_feature = df[576:]
# test label
test_label = label[576:]

# store all active ANNs
networks = []
pool = []
# Generation counter
generation = 0
# Initial Population
population = 10
for i in range(population):
    networks.append(ANN())
# Track Max Fitness
max_fitness = 0
# Store Max Fitness Weights
optimal_weights = []

epochs = 10
# Evolution Loop
for i in range(epochs):
    generation += 1
    logging.debug("Generation: " + str(generation) + "\r\n")

    for ann in networks:
        # Propagate to calculate fitness score
        ann.forward_propagation(train_feature, train_label)
        # Add to pool after calculating fitness
        pool.append(ann)

    # Clear for propagation of next children
    networks.clear()

    # Sort anns by fitness
    pool = sorted(pool, key=lambda x: x.fitness)
    pool.reverse()

    # Find Max Fitness and Log Associated Weights
    for i in range(len(pool)):
        if pool[i].fitness > max_fitness:
            max_fitness = pool[i].fitness

            logging.debug("Max Fitness: " + str(max_fitness) + "\r\n")

            # Iterate through layers, get weights, and append to optimal
            optimal_weights = []
            for layer in pool[i].layers:
                optimal_weights.append(layer.get_weights()[0])
            logging.debug('optimal_weights: ' + str(optimal_weights)+"\r\n")

    # Crossover: top 5 randomly select 2 partners
    for i in range(5):
        for j in range(2):
            # Create a child and add to networks
            temp = crossover(pool[i], random.choice(pool))
            # Add to networks to calculate fitness score next iteration
            networks.append(temp)

# Create a Genetic Neural Network with optimal initial weights
ann = ANN(optimal_weights)
predict_label = ann.predict(test_feature.values)
print('Test Loss: %.2f' % brier_score_loss(test_label, predict_label.round()))
print('Max Fitness: %.2f' % max_fitness)
print('Test Accuracy: %.2f' % accuracy_score(test_label, predict_label.round()))

2022-01-07 10:33:41,675 Generation: 1

2022-01-07 10:33:42,235 Max Fitness: 0.7441860465116279

2022-01-07 10:33:42,238 optimal_weights: [array([[ 0.13603479,  0.5427541 , -0.43404847, -0.5123948 ,  0.6030039 ,
        -0.5849414 , -0.46906245,  0.13151652],
       [-0.08255917,  0.4236849 ,  0.56392235,  0.2022956 , -0.41092223,
        -0.50589615, -0.05224627,  0.5120488 ],
       [ 0.4755903 ,  0.11499125, -0.20663342, -0.46338826, -0.5820237 ,
         0.28209662, -0.55730337,  0.19160116],
       [ 0.5532089 , -0.42803207,  0.4314112 ,  0.2314946 ,  0.53684133,
         0.37301975, -0.53774613,  0.0473119 ],
       [ 0.18955189,  0.02856994,  0.4545229 ,  0.29337305,  0.49023694,
        -0.1120407 , -0.10124892, -0.4673665 ],
       [ 0.25819337,  0.42300683,  0.12505275,  0.3855065 ,  0.5261857 ,
        -0.06209058,  0.3808856 , -0.1804559 ],
       [ 0.50585765, -0.17503136,  0.56346244, -0.49886167,  0.1426146 ,
         0.38308394, -0.42843372, -0.20645297],
       [ 0.2448

Test Loss: 0.36
Max Fitness: 0.74
Test Accuracy: 0.64
